In [ ]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [9]:
import os
from zipfile import ZipFile

!wget "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip"

dir = os.getcwd() + '/happy-or-sad.zip'

zipDir = ZipFile(dir,'r')
zipDir.extractall('/tmp/h-or-sad/')
zipDir.close()

!rm happy-or-sad.zip*

In [24]:
import tensorflow.keras as keras
from tensorflow.keras import layers

model = keras.Sequential([
  layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(32, (3,3), activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(32, (3,3), activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9248)             

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

imageGenerator = ImageDataGenerator(rescale=1/255)

image = imageGenerator.flow_from_directory(
    '/tmp/h-or-sad/',
    target_size=(150,150),
    batch_size=5,
    class_mode='binary',
)


Found 80 images belonging to 2 classes.


In [27]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    optimizer=RMSprop(lr=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [30]:
ACCURACY = 0.999

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > ACCURACY:
      print('\nReached',ACCURACY*100,'\b% Accuracy')
      self.model.stop_training = True

callback = myCallback()

In [32]:
history = model.fit_generator(
    image,
    steps_per_epoch=16,
    epochs=20,
    verbose=1,
    callbacks=[callback]
)

Epoch 1/20
16/16 [==============================] - ETA: 0s - loss: 0.0015 - accuracy: 1.0000
Reached 99.9 % Accuracy
16/16 [==============================] - 3s 214ms/step - loss: 0.0015 - accuracy: 1.0000


In [42]:
print('Accuracy:',history.history.get('accuracy')[0])
print('Loss:',history.history.get('loss')[0])

Accuracy: 1.0
Loss: 0.0015384296420961618
